In [3]:
from dcicutils import ff_utils
from functions.notebook_functions import get_key

# get key from file
my_key = get_key('koray_data')

# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# get admin key from s3
#my_key = ff_utils.get_authentication_with_server({}, ff_env = 'data')

In [5]:
q = '/search/?status=released&status=released+to+project&status=archived&type=WorkflowRun'
all_wfrs = ff_utils.search_metadata(q, key=my_key)

In [6]:
print(len(all_wfrs))

8077


In [19]:

problematic = []
safe = []

# try test status or rel, rel to pro, and arch
test_status = 'archived'
for a_wfr in all_wfrs:
    if a_wfr['status'] == test_status:
        wf_info = a_wfr.get('workflow')
        if wf_info['uuid'] not in problematic:
            if wf_info['uuid'] not in safe:
                wf_resp = ff_utils.get_metadata(wf_info['uuid'], key = my_key)
                if wf_resp['status'] not in ['released to project','released', test_status]:
                    problematic.append(wf_info['uuid'])
                else:
                    safe.append(wf_info['uuid'])
print(len(problematic))

1


In [40]:
mylist = ['bef50397-4d72-4ed1-9c78-100e14e5c47f',
          'c19ee11e-9d5a-454f-af50-600a0cf990b6',
          '05b62bba-7bfa-46cc-8d8e-3d37f4feb8bd',
          '146da22a-502d-4500-bf57-a7cf0b4b2364',
          'bef50397-4d72-4ed1-9c78-100e14e5c47f',
          'bd6e25ea-f368-4758-a821-d30e0b5a4100']

for wf in mylist:
    wf_data = ff_utils.get_metadata(wf, key=my_key)
    print(wf_data['display_title'], wf_data['status'])

File Provenance Tracking Workflow - 4DNWFBFZYHIE released
Generation of multiresolution Hi-C contact matrices from a set of contact lists, no restriction enzyme - 4DNWFQVDVA37 released
Generation of multiresolution Hi-C contact matrices from a set of contact lists, no normalization, no restriction enzyme - 4DNWFYACVAEX released
Repli-seq data processing pipeline for alignment, filtering and counting - 4DNWFBKH3JR1 released
File Provenance Tracking Workflow - 4DNWFBFZYHIE released
Generation of multiresolution Hi-C contact matrices from a set of contact lists, no normalization - 4DNWF7WTHVSA released


In [41]:
from functions.notebook_functions import *
schema_name = get_schema_names(my_key)
store, uuids = record_object_es(mylist,my_key, schema_name, store_frame='object')

In [42]:
for a_key in store:
    print(a_key)
    for i in store[a_key]:
        if i['status'] not in ['current', 'released']:
            print(i['status'], i['display_title'])
            ff_utils.patch_metadata({'status':'released'}, obj_id=i['uuid'], key=my_key)
            
            
    print()

lab

award

software

user

workflow

